In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from config import g_key
gmaps.configure(api_key=g_key)

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,punta arenas,CL,-53.150002,-70.916672,1569207600,90,84,43.97,11.34
1,chokurdakh,RU,70.633331,147.916672,1569207600,0,61,36.52,9.82
2,marienburg,SR,5.846910,-55.021591,1569207600,58,82,82.31,19.17
3,bernardino de campos,BR,-23.013060,-49.474171,1569207600,60,70,59.09,19.77
4,yantal,RU,56.847099,105.254204,1569207600,87,62,62.47,4.68


In [3]:
city_data_df.dtypes

City           object
Country        object
Lat           float64
Lng           float64
Date            int64
Cloudiness      int64
Humidity        int64
Max Temp      float64
Wind Speed    float64
dtype: object

In [4]:
humidity = city_data_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = city_data_df[["Lat", "Lng"]]

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & (city_data_df["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,morondava,MG,-20.283331,44.283329,1569207600,0,60,75.83,8.30
1,nelson bay,AU,-32.716671,152.149994,1569207600,0,54,71.58,8.72
2,santa vitoria,BR,-18.838610,-50.121391,1569207600,0,58,77.99,14.14
3,seoul,KR,37.568260,126.977829,1569207600,0,53,72.64,12.66
4,sao felix do xingu,BR,-6.644720,-51.994999,1569207600,0,64,75.65,1.74


In [7]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,morondava,MG,-20.283331,44.283329,1569207600,0,60,75.83,8.30,Morondava
1,nelson bay,AU,-32.716671,152.149994,1569207600,0,54,71.58,8.72,Nelson Bay - Corlette
2,santa vitoria,BR,-18.838610,-50.121391,1569207600,0,58,77.99,14.14,Santa Vitória
3,seoul,KR,37.568260,126.977829,1569207600,0,53,72.64,12.66,Seoul
4,sao felix do xingu,BR,-6.644720,-51.994999,1569207600,0,64,75.65,1.74,São Félix do Xingu


In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed
2,marienburg,SR,5.846910,-55.021591,1569207600,58,82,82.31,19.17
8,hithadhoo,MV,-0.600000,73.083328,1569207600,95,74,81.95,14.97
11,khon buri,TH,14.525410,102.245911,1569207600,100,97,76.19,4.09
13,arlit,NE,18.736940,7.385280,1569207600,0,12,86.81,9.95
15,bathsheba,BB,13.216670,-59.516670,1569207600,100,78,81.77,21.79
19,bengkulu,ID,-3.800440,102.265541,1569207600,99,89,76.73,17.36
22,morondava,MG,-20.283331,44.283329,1569207600,0,60,75.83,8.30
25,kavieng,PG,-2.574440,150.796692,1569207600,100,70,83.03,2.15
28,kousseri,CM,12.076890,15.030630,1569207600,60,71,79.61,6.85
37,atuona,PF,-9.800000,-139.033325,1569207600,14,78,80.29,18.59


In [10]:
preferred_cities_df.count()


City          140
Country       140
Lat           140
Lng           140
Date          140
Cloudiness    140
Humidity      140
Max Temp      140
Wind Speed    140
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,marienburg,SR,82.31,5.846910,-55.021591,
8,hithadhoo,MV,81.95,-0.600000,73.083328,
11,khon buri,TH,76.19,14.525410,102.245911,
13,arlit,NE,86.81,18.736940,7.385280,
15,bathsheba,BB,81.77,13.216670,-59.516670,
19,bengkulu,ID,76.73,-3.800440,102.265541,
22,morondava,MG,75.83,-20.283331,44.283329,
25,kavieng,PG,83.03,-2.574440,150.796692,
28,kousseri,CM,79.61,12.076890,15.030630,
37,atuona,PF,80.29,-9.800000,-139.033325,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))